## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import re
import sys
import json
import pathlib as pl
sys.path.append('../Core')
from StreamStats_API_Scraper import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

##### Specify:

In [2]:
#The state abbreviation in uppercase
state = 'NY' 

#Specify the location of the shapefile/json containing the lat/lon of points on the stream grid
path = pl.Path(r'P:\Temp\jMatney\R2_RII\json')
name = 'riv_gauges_parsed.json' #The name of the shapefile/json

#Specify a consistent coordinate reference system
use_epsg = '4326' 

#Location to save the StreamStats results for each polygon
allresults = os.path.join(path,'AllStreamStats') 
if os.path.isdir(allresults) == False: 
    os.mkdir(allresults)  

### Load the json:

In [3]:
with open(path/name) as f:
    dic = json.load(f)
print('{0} gages'.format(len(dic['features'])))    
dic['features'][0]

299 gages


{'type': 'Feature',
 'id': 0,
 'geometry': {'type': 'Point', 'coordinates': [-75.33027778, 39.64388889]},
 'properties': {'FID': 0,
  'State': 'NJ',
  'Gauge_Name': 'Salem River At Woodstown Nj',
  'X': 39.643889,
  'Y': -75.330278,
  'Gauge_No': '01482500'}}

### Select for state of interest:

In [4]:
select = {}
for feature in dic['features']:
    if feature['properties']['State'] == state:
        select[feature['properties']['Gauge_No']] = feature['geometry']['coordinates']
print('{0} gages for {1}'.format(len(list(select.keys())), state))  

159 gages for NY


##### Initialize parameters

In [5]:
#Dictionary to store the catchment polygons (catchment boundaries) 
polyg = {} 

#Dictionary to store the outlet flow frequency data dictionaries
ffdata = {} 

get_flow = False
print_status = True
if state=='WI': 
    get_flow = False 

### Run the API tool for each point:

In [ ]:
errors = []
for ID_Num, geom in select.items():
    try:
        #Longitude and latitude for each shapely point and the confluence number
        lon, lat = geom[0], geom[1]
        if print_status: 
            print("Lat/Lon/Confluence:", lat, lon, ID_Num)
            
        #Run the SS_scrape function. Option: set status=False to hide print statements
        polyg[ID_Num], ff_json  = SS_scrape(state, lon, lat, use_epsg, print_status) 
        if get_flow: 
            #Use the function above to extract the json data
            ffdata[ID_Num]                           = get_peaks(ff_json) 
            polyg[ID_Num]['features'][0]['ffcurve']  = ffdata[ID_Num]
        
        with open(os.path.join(allresults,'StreamStats_Polygons_{0}.geojson'.format(int(ID_Num))), 'w') as f:
            dump(polyg[ID_Num], f)
    except:
        print('could not process data {}'.format(ID_Num))
        errors.append(ID_Num) 

Lat/Lon/Confluence: 42.03527778 -75.8030556 01503000


### Load the results:

In [ ]:
files = load_files(allresults)
gdf2, ffdic=load_results(files, use_epsg)

In [ ]:
gdf.drop(gdf.loc[gdf['Id'][gdf2['ID_Num']]].index, inplace=True)
gdf.index.values

###  Save:

##### The flow frequency data as a CSV:

In [ ]:
#Run this function to construct the summary table for all outlet locations
if get_flow: ff_df=ff_summary(ffdic) 
    
#Save the results as a csv
if get_flow: ff_df.to_csv(os.path.join(path,'StreamStats_FlowFrequency.csv')) 

##### The catchment polygons as a Shapefile:

In [ ]:
#Export the geodataframe as a shapefile
gdf2 = convert_attr(gdf2)
gdf2.to_file(filename = os.path.join(path,'StreamStats_Polygons.shp')) 

##### The catchment polygons as a geojson:

In [ ]:
with open(os.path.join(path,'StreamStats_Polygons.geojson'), 'w') as f:
     dump(gdf2, f)  

# END